In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle

from gpu import set_gpu
import copy as cp
from sklearn.utils import shuffle

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from DL_utils import MLP, MLPbeta, CNN, eval_nn, train_mlp, test_mlp, train_mlpbeta, test_mlpbeta, train_cnn, test_cnn
import session_new as session

import process_data as prd
from lda import train_lda, predict, eval_lda, eval_lda_ch

set_gpu()

Using TensorFlow backend.


Num GPUs Available:  1


In [2]:
sub_type = 'AB'
with open('train_data_raw_'  + sub_type + '.p', 'rb') as f:
    raw, params,_,_ = pickle.load(f)

n_subs = np.max(params[:,0])

In [4]:
mlp = MLP()
mlp_beta = MLPbeta()
cnn = CNN()

mlp(x_train_mlp)
mlp_beta(x_train_mlp)
cnn(x_train_cnn)

mlp.save_weights('mlp_init_2.h5')
mlp_beta.save_weights('mlpbeta_init_2.h5')
cnn.save_weights('cnn_init_2.h5')

In [3]:
train_sess = {'sub_type':sub_type,'train':'fullallmix4', 'train_grp':2, 'train_scale':5, 'cv_type':'all','scaler_load':False,'feat_type':'feat','epochs':30}
test_sess = {'test_grp':4, 'test':'partposrealmixeven24'}
sess = session.Sess(**train_sess)
sess.update(**test_sess)
n_subs = 4#np.max(params[:,0])

for sub in range(3,n_subs+1):
    sess.sub = sub
    # tf.keras.backend.clear_session()

    if sub > 3:
        mlp.load_weights('mlp_init.h5')
        mlp_beta.load_weights('mlpbeta_init.h5')
        cnn.load_weights('cnn_init.h5')
    else:
    # Train NNs
        mlp = MLP()
        mlp_beta = MLPbeta()
        cnn = CNN()

        loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
        optimizer = tf.keras.optimizers.Adam()

        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

        test_loss = tf.keras.metrics.Mean(name='test_loss')
        test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

    # mlp.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
    # mlp_beta.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
    # cnn.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

    train_ind = (params[:,0] == sess.sub) & (params[:,3] == sess.train_grp)
    if np.sum(train_ind):
        trainmlp, validmlp, traincnn, validcnn, y_train, y_valid, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, x_train_aug, y_train_aug = prd.prep_train_data(sess,raw,params)
        with open('noisedata_all_emgscalelim/AB' + str(sub) + '_grp_2_fullallmix4_5.p','rb') as f:
            scaler, x_train_cnn, _, _, _, y_train, _, x_train_lda, y_train_lda, x_train_aug, y_train_aug = pickle.load(f)
        
        x_train_cnn = x_train_cnn.astype('float32')
        x_train_mlp = x_train_cnn.reshape(x_train_cnn.shape[0],-1)
        
        trainmlp = tf.data.Dataset.from_tensor_slices((x_train_mlp, y_train)).shuffle(x_train_mlp.shape[0],reshuffle_each_iteration=True).batch(128)
        traincnn = tf.data.Dataset.from_tensor_slices((x_train_cnn, y_train)).shuffle(x_train_cnn.shape[0],reshuffle_each_iteration=True).batch(128)
        
        sess.scaler = scaler

        models = ['mlp','mlpbeta','cnn']

        # mlp.fit(x_train_mlp,y_train,epochs=30,batch_size=128,verbose=0)
        # mlp_beta.fit(x_train_mlp,y_train,epochs=30,batch_size=128,verbose=0)
        # cnn.fit(x_train_cnn,y_train,epochs=30,batch_size=128,verbose=0)

        for model in models:
            print('Training ' + model)
            for epoch in range(sess.epochs):
                # Reset the metrics at the start of the next epoch
                train_loss.reset_states()
                train_accuracy.reset_states()
                test_loss.reset_states()
                test_accuracy.reset_states()

                # Train MLP
                if model == 'mlp':
                    for x, y in trainmlp:
                        train_mlp(x, y, mlp, loss_fn, optimizer, train_loss, train_accuracy)
                # Train MLP Beta
                elif model == 'mlpbeta':
                    for x, y in trainmlp:
                        train_mlpbeta(x, y, mlp_beta, loss_fn, optimizer, train_loss, train_accuracy)
                # Train CNN
                elif model == 'cnn':
                    for x, y in traincnn:
                        train_cnn(x, y, cnn, loss_fn, optimizer, train_loss, train_accuracy)

                if epoch == 0 or epoch == sess.epochs-1:
                    print(
                        f'Epoch {epoch + 1}, '
                        f'Loss: {train_loss.result():.2f}, '
                        f'Accuracy: {train_accuracy.result() * 100:.2f} '
                    )
        
        # Train aligned
        y_train_aligned = np.argmax(y_train, axis=1)[...,np.newaxis]

        mlp_enc = mlp.get_layer(name='enc')
        w_mlp, c_mlp,_, _, _ = train_lda(mlp_enc(x_train_mlp).numpy(),y_train_aligned)

        mlpbeta_enc = mlp_beta.get_layer(name='enc')
        w_mlpbeta, c_mlpbeta,_, _, _ = train_lda(mlpbeta_enc(x_train_mlp).numpy(),y_train_aligned)
        
        cnn_enc = cnn.get_layer(name='enc')
        temp = cnn_enc(x_train_cnn[:x_train_cnn.shape[0]//2,...]).numpy()
        temp2 = np.vstack((temp,cnn_enc(x_train_cnn[x_train_cnn.shape[0]//2:,...]).numpy()))
        w_cnn, c_cnn,_, _, _ = train_lda(temp2,y_train_aligned)

        # Train LDA
        w,c, _, _, _ = train_lda(x_train_lda,y_train_lda)
        w_aug,c_aug, _, _, _ = train_lda(x_train_aug,y_train_aug)

        # Load test data
        with open('real_noise/all_real_noise.p', 'rb') as f:
            real_noise_temp, _ = pickle.load(f)

        test_ind = (params[:,0] == sess.sub) & (params[:,3] == sess.test_grp)

        if np.sum(test_ind):
            x_test_cnn, x_test_mlp, x_test_lda, y_test, clean_size = prd.prep_test_data(sess, raw, params, real_noise_temp)
            with open('testdata_all_emgscalelim_noisescalelim/AB' + str(sub) + '_grp_2_partposrealmixeven14_1.p','rb') as f:
                x_test_cnn, _, x_test_lda, y_test = pickle.load(f) 
            x_temp = np.transpose(x_test_lda.reshape((x_test_lda.shape[0],4,-1)),(0,2,1))[...,np.newaxis]
            x_test_cnn = scaler.transform(x_temp.reshape(x_temp.shape[0]*x_temp.shape[1],-1)).reshape(x_temp.shape)
            x_test_cnn = x_test_cnn.astype('float32')

            # Reshape for nonconvolutional SAE
            x_test_mlp = x_test_cnn.reshape(x_test_cnn.shape[0],-1)
            # Test
            mlp_test_aligned = mlp_enc(x_test_mlp).numpy()
            mlpbeta_test_aligned = mlpbeta_enc(x_test_mlp).numpy()
            cnn_test_aligned = cnn_enc(x_test_cnn).numpy()
            y_test_aligned = np.argmax(y_test, axis=1)[...,np.newaxis]

            clean_lda = eval_lda(w, c, x_test_lda[:clean_size,...], y_test_aligned[:clean_size,...])
            noisy_lda = eval_lda(w, c, x_test_lda[clean_size:,...], y_test_aligned[clean_size:,...])
            print(
                f'LDA ---- '
                f'Clean: {clean_lda * 100:.2f}, '
                f'Noisy: {noisy_lda * 100:.2f}'
            )

            clean_lda = eval_lda(w_aug, c_aug, x_test_lda[:clean_size,...], y_test_aligned[:clean_size,...])
            noisy_lda = eval_lda(w_aug, c_aug, x_test_lda[clean_size:,...], y_test_aligned[clean_size:,...])
            print(
                f'AUG ---- '
                f'Clean: {clean_lda * 100:.2f}, '
                f'Noisy: {noisy_lda * 100:.2f}'
            )

            clean_acc, noisy_acc = eval_nn(x_test_mlp, y_test,mlp,clean_size)
            clean_lda = eval_lda(w_mlp,c_mlp,mlp_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
            noisy_lda = eval_lda(w_mlp,c_mlp,mlp_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
            print(
                f'MLP ---- '
                f'Clean: {clean_acc * 100:.2f}, '
                f'Noisy: {noisy_acc * 100:.2f}, '
                f'LDA Clean: {clean_lda * 100:.2f}, '
                f'LDA Noisy: {noisy_lda * 100:.2f}'
            )

            clean_acc, noisy_acc = eval_nn(x_test_mlp, y_test,mlp_beta,clean_size)
            clean_lda = eval_lda(w_mlpbeta,c_mlpbeta,mlpbeta_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
            noisy_lda = eval_lda(w_mlpbeta,c_mlpbeta,mlpbeta_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
            print(
                f'MLPB ---- '
                f'Clean: {clean_acc * 100:.2f}, '
                f'Noisy: {noisy_acc * 100:.2f}, '
                f'LDA Clean: {clean_lda * 100:.2f}, '
                f'LDA Noisy: {noisy_lda * 100:.2f}'
            )

            clean_acc, noisy_acc = eval_nn(x_test_cnn, y_test,cnn,clean_size)
            clean_lda = eval_lda(w_cnn,c_cnn,cnn_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
            noisy_lda = eval_lda(w_cnn,c_cnn,cnn_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
            print(
                f'CNN ---- '
                f'Clean: {clean_acc * 100:.2f}, '
                f'Noisy: {noisy_acc * 100:.2f}, '
                f'LDA Clean: {clean_lda * 100:.2f}, '
                f'LDA Noisy: {noisy_lda * 100:.2f}'
            )
        else:
            print('no testing data')
    else:
        print('no training data')



Loading training data: traindata_all/AB3_traindata_2.p
Training mlp
Epoch 1, Loss: 1.80, Accuracy: 30.89, 
Epoch 30, Loss: 0.56, Accuracy: 80.22, 
Training mlpbeta
Epoch 1, Loss: 1.01, Accuracy: 65.99, 
Epoch 30, Loss: 0.19, Accuracy: 93.43, 
Training cnn
Epoch 1, Loss: 0.89, Accuracy: 71.91, 
Epoch 30, Loss: 0.15, Accuracy: 94.57, 
Loading training data: traindata_all/AB3_traindata_4.p
LDA ---- Clean: 76.14, Noisy: 60.57
AUG ---- Clean: 57.60, Noisy: 51.94
MLP ---- Clean: 70.86, Noisy: 69.83, LDA Clean: 74.26, LDA Noisy: 73.20
MLPB ---- Clean: 80.97, Noisy: 79.63, LDA Clean: 79.91, LDA Noisy: 79.74
CNN ---- Clean: 78.26, Noisy: 77.03, LDA Clean: 74.94, LDA Noisy: 74.03
Loading training data: traindata_all/AB4_traindata_2.p
Training mlp
Epoch 1, Loss: 1.78, Accuracy: 27.68, 
Epoch 30, Loss: 0.79, Accuracy: 69.87, 
Training mlpbeta
Epoch 1, Loss: 1.36, Accuracy: 50.17, 
Epoch 30, Loss: 0.36, Accuracy: 86.41, 
Training cnn
Epoch 1, Loss: 1.38, Accuracy: 47.91, 
Epoch 30, Loss: 0.32, Accu